<a href="https://colab.research.google.com/github/suresh/notebooks/blob/master/evidence_inspector_release.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

In [ ]:
!git clone https://github.com/kukrishna/evidence_inspector.git

In [ ]:
%cd /content/evidence_inspector/
!pip install -r requirements.txt

In [ ]:
!python -m spacy download en_core_web_md

In [ ]:
%cd /content/evidence_inspector/backend/
!nohup python serve_seq2seq_simple.py --model-path kundank/evinspect-usb-flant5-large --port 9922  > /content/nohup_backend.out &

# alternatively you can run the larger model, but would probably not fit in the free colab T4 gpu
# !nohup python serve_seq2seq_qlora.py --model-path kundank/evinspect-usb-flanul2-qlora4bit --port 9922  > /content/nohup_backend.out &

In [ ]:
# The following optinal command would run a backend LLM to do QA against the document in the interface. We use a falcon-7b model by default but others should work too
# However, the free Colab GPU instance does not have enough memory to run both the fact-checking model and this LLM for QA.
# You can use the fact-checking features of the interface even without running this. If you try to use the QA actions in the UI you would simply get the error "QA model not reachable...".

# %cd /content/evidence_inspector/backend/
# !nohup python serve_generic_llm.py  --model-name tiiuae/falcon-7b-instruct --port 9003 --quantize 16bit --nbeams 4  > /content/nohup_backend_llm.out &

In [ ]:
# Since the colab kernel is not directly accessible through internet, we will tunnel the traffic through an intermediary (serveo.net)
# !! DO NOT PUT PRIVATE OR SENSITIVE DATA IN THE INTERFACE. WE TAKE NO RESPONSIBILITY FOR THE WAY THE DATA IS USED (IF ANY) BY THE INTERMEDIARY !!

!nohup ssh -o StrictHostKeyChecking=no -R 80:localhost:80 serveo.net > /content/nohup_pf.out &
!sleep 10

In [ ]:
# Remember that opening 127.0.0.1 on your browser won't show up this tool. You need to open the URL printed by this cell to access it

!echo "When you run the last cell and the server starts serving, open the URL given below in a new tab to access the system."
!cat /content/nohup_pf.out | grep "Forwarding HTTP"

In [ ]:
# the backend model is probably still getting downloaded at this point. This should show the progress
!cat /content/nohup_backend.out | grep "Downloading pytorch_model.bin"

In [ ]:
# before starting the frontend, the backend must download and serve the model. the frontend will wait for the backend to start.
# if it is taking too long, you can stop this cell and run the previous cell to  see the download progress:

# Also recall that you will NOT be able to access the tool using the http://127.0.0.1 URL printed by this cell.
# Instead, to access the tool, go to the <something>.serveo.net URL that was printed by an earlier cell

%cd /content/evidence_inspector/frontend
!python job_server.py --port 80